In [ ]:
"""
TO DO LIST:
    - evaluate the default algorithm
        - holdout
        - confusion matrix
        - ROC / AUC
    
    - compare the default algorithm with the modified ones for single task (my dataset)
        - get each modified algorithm houldout accuracy and standard deviation (SD) estimates
        - calculate the difference in accuracy and SD between the default model and each of the mdified ones
        - Are the differences statistically significant? -> Run hypothesis tests (paired t-test or wilcoxon)

TO LATER DO WITH GROUP:
    - 
        


"""

In [1]:
# imports

import pandas as pd
import numpy as np
import openml
from AdaBoostWorkyWorky import AdaBoost
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


In [2]:
# get the data from OpenML
# dataset from Task ID = 3904

suite = openml.study.get_suite(99)
task_id = 3904
task = openml.tasks.get_task(task_id)    
dataset = openml.datasets.get_dataset(task.dataset_id)

X, y, categorical_indicator, attribute_names = dataset.get_data(
    dataset_format="array", target=dataset.default_target_attribute
)

# Visualizar o dataset

df = pd.DataFrame(X, columns=attribute_names)
# converter:
#   0 -> -1
#   1 -> 1
df['target'] = 2*y-1 
df = df.dropna(how='any', axis=0)

X = df.drop(columns=['target'], axis=1)
y = df.target

df

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,target
0,1.1,1.4,1.4,1.4,1.3,1.300000,1.30,1.300000,1.300000,1.300000,...,2.0,2.0,2.0,2.0,1.2,1.2,1.2,1.2,1.4,-1
1,1.0,1.0,1.0,1.0,1.0,1.000000,1.00,1.000000,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
2,72.0,7.0,1.0,6.0,198.0,1134.130005,0.05,20.309999,55.849998,23029.099609,...,51.0,10.0,8.0,1.0,17.0,36.0,112.0,86.0,13.0,1
3,190.0,3.0,1.0,3.0,600.0,4348.759766,0.06,17.059999,254.869995,74202.671875,...,129.0,29.0,28.0,2.0,17.0,135.0,329.0,271.0,5.0,1
4,37.0,4.0,1.0,4.0,126.0,599.119995,0.06,17.190001,34.860001,10297.299805,...,28.0,1.0,6.0,0.0,11.0,16.0,76.0,50.0,7.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10880,18.0,4.0,1.0,4.0,52.0,241.479996,0.14,7.330000,32.930000,1770.859985,...,13.0,0.0,2.0,0.0,10.0,15.0,30.0,22.0,7.0,-1
10881,9.0,2.0,1.0,2.0,30.0,129.660004,0.12,8.250000,15.720000,1069.680054,...,5.0,0.0,2.0,0.0,12.0,8.0,19.0,11.0,3.0,-1
10882,42.0,4.0,1.0,2.0,103.0,519.570007,0.04,26.400000,19.680000,13716.719727,...,29.0,1.0,10.0,0.0,18.0,15.0,59.0,44.0,7.0,-1
10883,10.0,1.0,1.0,1.0,36.0,147.149994,0.12,8.440000,17.440001,1241.569946,...,6.0,0.0,2.0,0.0,9.0,8.0,21.0,15.0,1.0,-1


In [3]:
# splitting the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1111)

In [11]:
# holdout estimation function

def holdout_estimation(model, x, y, test_size_value=0.3, seed=1111):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size_value, random_state=seed)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(f"Estimated accuracy by holdout {accuracy_score(y_test, y_pred):.5f}, using {model.__class__.__name__}")

In [12]:
# fit and predict using the implementation of the default AdaBoost algorithm
# using cross-validation to get an average of the accuracy of the model

ab = AdaBoost()
holdout_estimation(ab, X, y)



Estimated accuracy by holdout 0.81556, using AdaBoost
